# Лабораторная работа

Давайте убедимся, что необходимые нам для этой работы библиотеки уже установлены.

In [ ]:
!pip install -q numpy tensorflow_datasets tensorflow

# Описание набора данных
В данной работе целью исследования будет набор данных Titanic. Данный набор содержит информацию о различных пассажирах корабля "Титаник", затонувшего в ночь на 15-е апреля 1912 года. Некоторое количество пассажиров спаслось, чему способствовало множество различных факторов, включая их пол, возраст, на какой палубе находилась их кабина, социальный статус, и т.д. Мы предлагаем вам натренировать нейронную сеть для бинарной классификации, способную предсказывать вероятность спасения человека на основе его данных.  

Набор данных состоит из различных признаков, описывающих информацию о пассажирах. Каждая строка таблицы - отдельный пассажир, вся информация о нем содержится в его строке. В столбце survived находится бинарная метка (0 или 1), означающая, спасся ли человек с корабля (1) или нет (0). Вашей задачей является изучить набор данных, корректно определить решаемую задачу, а затем построить модель, которая будет способна предсказывать вероятность спасения и посчитать F1-метрику на тестовом наборе данных.

Для загрузки набора данных мы будем использовать библиотеку tensorflow_datasets. 

Импортируем данную библиотеку, а затем с помощью нее загрузим набор данных "titanic". В процессе загрузки сразу разделим его на тренировочную, валидационную и тестовую выборку. Еще одну часть датасета мы зарезеривировали для оценки вашей модели  системой проверки на образовательной платформе.

In [1]:
import tensorflow_datasets as tfds
ds_train_tf, ds_validation_tf, ds_test_tf = tfds.load(
    name='titanic',
    split=['train[:70%]', 'train[70%:80%]', 'train[80%:90%]'],
    as_supervised=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/titanic/4.0.0.incomplete1PJ9HN/titanic-train.tfrecord*...:   0%|          …

Dataset titanic downloaded and prepared to /root/tensorflow_datasets/titanic/4.0.0. Subsequent calls will reuse this data.


По умолчанию библиотека загружает данные в формате TensorFlow Dataset. Этот формат позволяет оперировать данными на жестком диске без предварительной загрузки их в память, что позволят обучать модели на данных, превышающих размер оперативной памяти вашего устройства.

Набор данных Titanic является достаточно небольшим набором и мы уверены, что оперативной памяти вашего устройства хватит для хранения его целиком, поэтому это в данный момент не играет решающей роли. Однако, вам будет полезно научиться работать с ним.

Примеры использования и различных функций этого формата данных вы можете найти в документации или Jupyter Notebook'е данного курса, описывающем работу с библиотекой Keras.

Давайте выведем первую строку тренировочного набора данных для ознакомления.

In [2]:
for x, y in ds_train_tf.take(1):
    print(f"x={x}")
    print(f"y={y}")

x={'age': <tf.Tensor: shape=(), dtype=float32, numpy=30.0>, 'boat': <tf.Tensor: shape=(), dtype=string, numpy=b'Unknown'>, 'body': <tf.Tensor: shape=(), dtype=int32, numpy=-1>, 'cabin': <tf.Tensor: shape=(), dtype=string, numpy=b'Unknown'>, 'embarked': <tf.Tensor: shape=(), dtype=int64, numpy=2>, 'fare': <tf.Tensor: shape=(), dtype=float32, numpy=13.0>, 'home.dest': <tf.Tensor: shape=(), dtype=string, numpy=b'Sarnia, ON'>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'McCrie, Mr. James Matthew'>, 'parch': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'pclass': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'sex': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'sibsp': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'ticket': <tf.Tensor: shape=(), dtype=string, numpy=b'233478'>}
y=0


In [4]:
type(ds_train_tf)

tensorflow.python.data.ops.prefetch_op._PrefetchDataset

Как можно заметить, формат текущего датасета представляет собой кортеж из двух элементов - признаков и метки класса. Признаки организованы как словарь с ключами (названиями признаков) и значениями признаков. Данный формат можно использовать напрямую для метода model.fit, однако признаки, несомненно, требуют предобработки.

Итак, к настоящему моменту мы загрузили набор данных, разделили его на тренировочную, валидационную и тестовую выборку и готовы приступить к решению задачи. Вашим текущим заданием является получение максимального значения F1-метрики на тестовом наборе данных, используя все возможности по изучению и визуализации данных, предобработке данных, а также нейронную сеть, выполненную с помощью фреймворка TensorFlow (и его части - Keras).  

Для того, чтобы напомнить процесс подготовки данных и обучения модели, мы поместили несколько ячеек с комментариями ниже в предполагаемом порядке. Однако, вы можете использовать любой удобный вам процесс.  

В процессе исследования и тренировки запрещено использовать данные из ds_test кроме как для финальной оценки F1 метрики. 

После того, как вы добились максимального на ваш взгляд значения метрики F1 на тестовых данных, выполните предсказание для зарезервированной части датасета (ссылку на данные можно найти на странице с заданием). После чего введите полученные метки в поле ввода на платформе.

Данная лабораторная работа служит не для проверки ваших навыков, а для того, чтобы помочь вам вспомнить основы машинного обучения и работы с соответствующими библиотеками. Мы просим вас попытаться решить задачу самостоятельно, не используя доступную информацию из зарезервированного набора и не пытаясь подогнать модель под идеальную работу с конкретно данным набором, а также просим не использовать внешние источники данных и информацию из соответствующих соревнований на платформе Kaggle.

Вы можете найти для себя полезным пройти снова ноутбук с описанием работы фреймворка Keras и информацией о том, как его использовать. Также по данному фреймворку доступны многочисленные туториалы и примеры на его сайте. Напомним вам, что задача представляет собой бинарную классификацию, поэтому вам необходимо выбрать правильное количество нейронов для последнего слоя и функцию активации.

Успехов!

In [ ]:
# Предобработка данных

In [ ]:
# 1. Предобработка данных: заполнение пропущенных мест, приведение столбцов к единому виду, удаление nan/infinity значений

In [ ]:
# 2. Инженерия признаков: генерация или выделение наиболее интересных признаков, удаление ненужных

In [ ]:
# 3. Нормализация данных относительно среднего и медианы

In [ ]:
# Обучение модели

In [ ]:
# 1. Выбор модели или ансамбля моделей

In [ ]:
# 2. Поиск оптимальных гиперпараметров на validation наборе данных

In [ ]:
# Оценка модели

In [ ]:
# 1. Получение F1-метрики на тестовом наборе данных

In [ ]:
# Предсказание для данных из зарезервированной части 

In [ ]:
# 1. Отправка предсказанных меток через образовательную платформу